# Classify text with BERT

This fine-tuned BERT model is trained to classify tweets into 3 labels, 0 = "hate speech", 1 = "offensive language", 2 = "neither". 

Trained on https://github.com/t-davidson/hate-speech-and-offensive-language

testing dataset from https://github.com/hate-alert/HateXplain

## Loading in packages and initilizing constants

In [2]:
import os
import shutil
from datetime import datetime
import pandas as pd
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

from datasets import load_dataset
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

device = 'GPU:0' if tf.test.is_gpu_available()else "/CPU:0"

logs = "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tf.get_logger().setLevel('ERROR')

epochs = 2
batch_size = 16
seed = 42
test_train_ratio = 0.1

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


## Loading in datasets

hate_data holds the train and validaiton dataset from the hatespeech dataset

train_ds, val_ds, and test_ds are batched and shuffled tf.data.Dataset 

In [3]:
def label_to_logitlabel(i):
    out = [0.0,0.0,0.0]
    out[i] = 1.0
    return out

# reading in train and validaiton dataset
hate_data = pd.read_csv("datasets\hate speech tweets\labeled_data.csv").drop(columns=["Unnamed: 0", "count", "hate_speech", "offensive_language", "neither"]).rename(columns={'class' : 'label', "tweet":"text"})
print(f'{len(hate_data)} items in the dataset')

# for human readability
class_names = ["hate speech", "offensive language", "neither"]


x_train_ = tf.convert_to_tensor(hate_data.text.values)
y_train_ = tf.convert_to_tensor(list(map(label_to_logitlabel, hate_data.label.values)))


train_ds = tf.data.Dataset.from_tensor_slices((x_train_, y_train_)).shuffle(int(len(x_train_)),seed=seed, reshuffle_each_iteration=True).batch(batch_size, name="inputs")

# loading in testing data set

customer_data = load_dataset("hatexplain", split="test")
customer_data = customer_data.map(lambda e: {"label" : label_to_logitlabel(int(np.median(e["annotators"]["label"]))), "post_tokens" : ' '.join(e["post_tokens"])})


customer_data.set_format(type="tf", columns="post_tokens")
token_tensor = customer_data['post_tokens']
customer_data.set_format(type="tf", columns="label")
label_tensor = customer_data['label']

24783 items in the dataset


Reusing dataset hatexplain (C:\Users\Ahmad\.cache\huggingface\datasets\hatexplain\plain_text\1.0.0\df474d8d8667d89ef30649bf66e9c856ad8305bef4bc147e8e31cbdf1b8e0249)
Loading cached processed dataset at C:\Users\Ahmad\.cache\huggingface\datasets\hatexplain\plain_text\1.0.0\df474d8d8667d89ef30649bf66e9c856ad8305bef4bc147e8e31cbdf1b8e0249\cache-07267514b418c022.arrow


In [4]:
temp = len(list(train_ds.as_numpy_iterator()))
print(f'{temp} batches\nbatch size: {batch_size}\nsanity check: {batch_size*temp} == {len(hate_data)}')

1549 batches
batch size: 16
sanity check: 24784 == 24783


In [5]:
#@title Choose a BERT model to fine-tune

bert_model_name = 'bert_en_uncased_L-12_H-768_A-12'  #@param ["bert_en_uncased_L-12_H-768_A-12", "bert_en_cased_L-12_H-768_A-12", "bert_multi_cased_L-12_H-768_A-12", "small_bert/bert_en_uncased_L-2_H-128_A-2", "small_bert/bert_en_uncased_L-2_H-256_A-4", "small_bert/bert_en_uncased_L-2_H-512_A-8", "small_bert/bert_en_uncased_L-2_H-768_A-12", "small_bert/bert_en_uncased_L-4_H-128_A-2", "small_bert/bert_en_uncased_L-4_H-256_A-4", "small_bert/bert_en_uncased_L-4_H-512_A-8", "small_bert/bert_en_uncased_L-4_H-768_A-12", "small_bert/bert_en_uncased_L-6_H-128_A-2", "small_bert/bert_en_uncased_L-6_H-256_A-4", "small_bert/bert_en_uncased_L-6_H-512_A-8", "small_bert/bert_en_uncased_L-6_H-768_A-12", "small_bert/bert_en_uncased_L-8_H-128_A-2", "small_bert/bert_en_uncased_L-8_H-256_A-4", "small_bert/bert_en_uncased_L-8_H-512_A-8", "small_bert/bert_en_uncased_L-8_H-768_A-12", "small_bert/bert_en_uncased_L-10_H-128_A-2", "small_bert/bert_en_uncased_L-10_H-256_A-4", "small_bert/bert_en_uncased_L-10_H-512_A-8", "small_bert/bert_en_uncased_L-10_H-768_A-12", "small_bert/bert_en_uncased_L-12_H-128_A-2", "small_bert/bert_en_uncased_L-12_H-256_A-4", "small_bert/bert_en_uncased_L-12_H-512_A-8", "small_bert/bert_en_uncased_L-12_H-768_A-12", "albert_en_base", "electra_small", "electra_base", "experts_pubmed", "experts_wiki_books", "talking-heads_base"]

map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

BERT model selected           : https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3


## The preprocessing model

Text inputs need to be transformed to numeric token ids and arranged in several Tensors before being input to BERT. TensorFlow Hub provides a matching preprocessing model for each of the BERT models discussed above, which implements this transformation using TF ops from the TF.text library. It is not necessary to run pure Python code outside your TensorFlow model to preprocess text.

The preprocessing model must be the one referenced by the documentation of the BERT model, which you can read at the URL printed above. For BERT models from the drop-down above, the preprocessing model is selected automatically.

Note: You will load the preprocessing model into a [hub.KerasLayer](https://www.tensorflow.org/hub/api_docs/python/hub/KerasLayer) to compose your fine-tuned model. This is the preferred API to load a TF2-style SavedModel from TF Hub into a Keras model.

In [6]:
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)
bert_model = hub.KerasLayer(tfhub_handle_encoder)

## Define the model

simple fine-tuned model, with the preprocessing model, the selected BERT model, one Dense and a Dropout layer.

In [7]:
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(3, activation="sigmoid", name='classifier')(net)
  return tf.keras.Model(text_input, net)

In [8]:
text_test = ['this is such an amazing movie!']
classifier_model = build_classifier_model()
bert_raw_result = classifier_model(tf.constant(text_test))
#print(tf.sigmoid(bert_raw_result))
#print(bert_raw_result)

tf.Tensor([[0.63724303 0.6468651  0.68992186]], shape=(1, 3), dtype=float32)
tf.Tensor([[0.56341803 0.6052879  0.7997541 ]], shape=(1, 3), dtype=float32)


## Model training

You now have all the pieces to train a model, including the preprocessing module, BERT encoder, data, and classifier.

### Loss function

Since this is a binary classification problem and the model outputs a probability (a single-unit layer), you'll use `losses.BinaryCrossentropy` loss function.


In [9]:
loss = tf.keras.losses.BinaryCrossentropy()
metrics = [
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall'),
    tf.keras.metrics.Accuracy(name='recall'),
]

### Optimizer

For fine-tuning, let's use the same optimizer that BERT was originally trained with: the "Adaptive Moments" (Adam). This optimizer minimizes the prediction loss and does regularization by weight decay (not using moments), which is also known as [AdamW](https://arxiv.org/abs/1711.05101).

For the learning rate (`init_lr`), you will use the same schedule as BERT pre-training: linear decay of a notional initial learning rate, prefixed with a linear warm-up phase over the first 10% of training steps (`num_warmup_steps`). In line with the BERT paper, the initial learning rate is smaller for fine-tuning (best of 5e-5, 3e-5, 2e-5).

In [10]:
steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

### Loading the BERT model and training

Using the `classifier_model` you created earlier, you can compile the model with the loss, metric and optimizer.

In [11]:
classifier_model.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)

Note: training time will vary depending on the complexity of the BERT model you have selected.

In [12]:
tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 profile_batch = '1,100')

In [13]:
print(f'Training model with {tfhub_handle_encoder}')
with tf.device(device):
    history = classifier_model.fit(x=train_ds,
                                epochs=epochs,
                                steps_per_epoch=30,
                                callbacks = [tboard_callback])

Training model with https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3
Epoch 1/2
30/30 [==============================] - 52s 1s/step - loss: 0.7970 - precision: 0.3761 - recall: 0.9229
Epoch 2/2
30/30 [==============================] - 31s 1s/step - loss: 0.4840 - precision: 0.7266 - recall: 0.8083


In [14]:
%load_ext tensorboard
%tensorboard --logdir=logs

### Evaluate the model

Let's see how the model performs. Two values will be returned. Loss (a number which represents the error, lower values are better), and accuracy.

In [15]:
loss, accuracy = classifier_model.evaluate(test_ds)

print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

NameError: name 'test_ds' is not defined

### Plot the accuracy and loss over time

Based on the `History` object returned by `model.fit()`. You can plot the training and validation loss for comparison, as well as the training and validation accuracy:

In [16]:
history_dict = history.history
print(history_dict.keys())

acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)
fig = plt.figure(figsize=(10, 6))
fig.tight_layout()

plt.subplot(2, 1, 1)
# r is for "solid red line"
plt.plot(epochs, loss, 'r', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
# plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(epochs, acc, 'r', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

dict_keys(['loss', 'precision', 'recall'])


KeyError: 'accuracy'

In this plot, the red lines represent the training loss and accuracy, and the blue lines are the validation loss and accuracy.

## Export for inference

Now you just save your fine-tuned model for later use.

In [ ]:
dataset_name = 'hate_speech'
saved_model_path = './{}_bert_L-12_H-768_A-12'.format(dataset_name.replace('/', '_'))

classifier_model.save(saved_model_path, include_optimizer=False)

Let's reload the model, so you can try it side by side with the model that is still in memory.

In [ ]:
dataset_name = 'hate_speech'
saved_model_path = './{}_bert_L-12_H-768_A-12'.format(dataset_name.replace('/', '_'))

reloaded_model = tf.saved_model.load(saved_model_path)

Here you can test your model on any sentence you want, just add to the examples variable below.

In [ ]:
import collections

def print_my_examples(inputs, results):
  result_for_printing = \
    [f'input: {inputs[i]} : label: {class_names[results[i]]}'
                         for i in range(len(inputs))]
  print(*result_for_printing, sep='\n')
  print()


examples = customer_data.sample(1000).text.values

original_results = np.argmax(classifier_model.predict(examples), axis=-1)
print(collections.Counter(original_results))

print('Results from the model in memory:')
print_my_examples(examples, original_results)

If you want to use your model on [TF Serving](https://www.tensorflow.org/tfx/guide/serving), remember that it will call your SavedModel through one of its named signatures. In Python, you can test them as follows:

In [ ]:
serving_results = reloaded_model \
            .signatures['serving_default'](tf.constant(examples))

serving_results = tf.sigmoid(serving_results['classifier'])

print_my_examples(examples, serving_results)